In [21]:
import numpy as np
import pandas as pd
import geopandas as gpd

import geopy
from geopy.geocoders import Nominatim

In [22]:
columns_to_import = ['LocationID', 'StreetNo', 'StreetName', 'City', 'State',
       'Zip', 'Lat', 'Long', 'COUNTY']

In [23]:
kli_coded = pd.read_csv('../data/raw/BCC LOCS W CNTY 020524.csv', usecols=columns_to_import);

In [24]:
# Convert 'col1' to string
kli_coded['LocationID'] = kli_coded['LocationID'].astype(str) 

In [52]:
scm = pd.read_excel('../data/raw/2024-09-01 Standard Coverage Model - JR.xlsx', sheet_name= 'CVG Model');

In [53]:
scm['Ship To #'] = scm['Ship To #'].astype(str) 

In [54]:
kli_coded.dtypes

LocationID     object
StreetNo       object
StreetName     object
City           object
State          object
Zip            object
Lat           float64
Long          float64
COUNTY         object
dtype: object

In [55]:
scm.dtypes

Ship To #             object
Ship To Name          object
Parent #               int64
Parent Name           object
City                  object
                      ...   
Net Total $$$ 2PY    float64
Line 1                object
Line 2                object
Line 3                object
Zip Code              object
Length: 127, dtype: object

In [63]:
scm_us = scm[scm['Country'] == 'US']
scm_us = scm_us[scm_us['State'] != 'PR']

In [64]:
df2 = pd.merge(kli_coded, scm_us, how='right', left_on='LocationID', right_on='Ship To #')

In [65]:
df2

,LocationID,StreetNo,StreetName,City_x,State_x,Zip,Lat,Long,COUNTY,Ship To #,...,Net Feature $$$ 2PY,Constellation Units 2PY,Net Constellation $$$ 2PY,Net Cremation $$$ 2PY,Net BI $$$ 2PY,Net Total $$$ 2PY,Line 1,Line 2,Line 3,Zip Code
0,131453,8578,W 32ND AVE,WHEAT RIDGE,CO,80033,39.762100,-105.092974,Jefferson,131453,...,NaN,NaN,NaN,NaN,NaN,NaN,8578 W 32ND AVE,,,80033
1,152738,180,W 76TH ST,NEW YORK,NY,10023-8401,40.780966,-73.979184,New York,152738,...,8501.69,0.0,0.00,34595.47,0.0,327795.53,180 W 76TH ST,,,10023-8401
2,150905,517,PARK STREET,JACKSONVILLE,FL,32204-2930,30.321048,-81.677880,Duval,150905,...,0.00,0.0,0.00,0.00,0.0,806.52,517 PARK STREET,,,32204-2930
3,365386,3344,SW 8TH ST,MIAMI,FL,33135,25.764823,-80.250381,Miami-Dade,365386,...,74088.86,9.0,6867.88,5951.87,0.0,506619.67,3344 SW 8TH ST,,,33135
4,141923,3888,WORKMAN MILL RD,WHITTIER,CA,90601-1626,34.005110,-118.055965,Los Angeles,141923,...,361932.43,175.0,164522.51,26795.53,0.0,2175634.36,3888 WORKMAN MILL RD,,,90601-1626
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37130,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,162454,...,NaN,NaN,NaN,NaN,NaN,NaN,100 STENTON AVE,,,19462
37131,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,164907,...,NaN,NaN,NaN,NaN,NaN,NaN,PO BOX 3551,,,95927
37132,347527,5901,W JIM BILTON BLVD,SAINT GEORGE,SC,29477,33.194417,-80.593752,Dorchester,347527,...,NaN,NaN,NaN,NaN,NaN,NaN,5901 W JIM BILTON BLVD,,,29477
37133,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140451,...,NaN,NaN,NaN,NaN,NaN,NaN,225 N. GARFIELD AVENUE,,,91801


In [66]:
df2[~df2['Lat'].isna()].to_csv('../data/processed/geocoded-updated-locations.csv')

In [67]:
needs_geo = df2[df2['Lat'].isna()]

In [68]:
needs_geo = needs_geo.drop(['LocationID', 'StreetNo', 'StreetName', 'City_x', 'State_x',
       'Zip', 'Lat', 'Long', 'COUNTY'], axis=1)

In [70]:
pd.options.display.max_columns = None

In [71]:
needs_geo

,Ship To #,Ship To Name,Parent #,Parent Name,City_y,State_y,Country,Division,Region Code,Region Desc,Territory Cd,Rep Name,Status \n(C-Open;\n CO-Closed),Customer Group Code,Customer Group,Customer Segment,Program Sub-Group 2,Program Sub-Group 1,Customer Spectrum,Representative Type,NAR Code,NAR Manager,Std Industry Cd,Customer Type,Tot Dths,Burials,Adj. Burials,Opportunity,Program,Cost Center / CSC,CSC Desc,Serv Lvl,Delivery Frequency,Delivery Service Priority,Buyer Type,Burial Units Prior Mo,Net Burial $$$ Prior Mo,Metal Units Prior Mo,Wood Units Prior Mo,Feature Units Prior Mo,Net Feature $$$ Prior Mo,Constellation Units Prior Mo,Net Constellation $$$ Prior Mo,Net Cremation $$$ Prior Mo,Net BI $$$ Prior Mo,Net Total $$$ Prior Mo,Burial Units Prior Mo PY,Net Burial $$$ Prior Mo PY,Metal Units Prior Mo PY,Wood Units Prior Mo PY,Feature Units Prior Mo PY,Net Feature $$$ Prior Mo PY,Constellation Units Prior Mo PY,Net Constellation $$$ Prior Mo PY,Net Cremation $$$ Prior Mo PY,Net BI $$$ Prior Mo PY,Net Total $$$ Prior Mo PY,Burial Units YTD,Net Burial $$$ YTD,Metal Units YTD,Wood Units YTD,Feature Units YTD,Net Feature $$$ YTD,Constellation Units YTD,Net Constellation $$$ YTD,Net Cremation $$$ YTD,Net BI $$$ YTD,Net Total $$$ YTD,Burial Units PYTD,Net Burial $$$ PYTD,Metal Units PYTD,Wood Units PYTD,Feature Units PYTD,Net Feature $$$ PYTD,Constellation Units PYTD,Net Constellation $$$ PYTD,Net Cremation $$$ PYTD,Net BI $$$ PYTD,Net Total $$$ PYTD,Burial Units Rolling 12 mo,Net Burial $$$ Rolling 12 mo,Metal Units Rolling 12 mo,Wood Units Rolling 12 mo,Feature Units Rolling 12 mo,Net Feature $$$ Rolling 12 mo,Constellation Units Rolling 12 mo,Net Constellation $$$ Rolling 12 mo,Net Cremation $$$ Rolling 12 mo,Net BI $$$ Rolling 12 mo,Net Total $$$ Rolling 12 mo,Burial Units Rolling 12 mo PY,Net Burial $$$ Rolling 12 mo PY,Metal Units Rolling 12 mo PY,Wood Units Rolling 12 mo PY,Feature Units Rolling 12 mo PY,Net Feature $$$ Rolling 12 mo PY,Constellation Units Rolling 12 mo PY,Net Constellation $$$ Rolling 12 mo PY,Net Cremation $$$ Rolling 12 mo PY,Net BI $$$ Rolling 12 mo PY,Net Total $$$ Rolling 12 mo PY,Burial Units PY,Net Burial $$$ PY,Metal Units PY,Wood Units PY,Feature Units PY,Net Feature $$$ PY,Constellation Units PY,Net Constellation $$$ PY,Net Cremation $$$ PY,Net BI $$$ PY,Net Total $$$ PY,Burial Units 2PY,Net Burial $$$ 2PY,Metal Units 2PY,Wood Units 2PY,Feature Units 2PY,Net Feature $$$ 2PY,Constellation Units 2PY,Net Constellation $$$ 2PY,Net Cremation $$$ 2PY,Net BI $$$ 2PY,Net Total $$$ 2PY,Line 1,Line 2,Line 3,Zip Code
198,358501,"WEST COLORADO FUNERAL SERVICE, LLC",358501,"WEST COLORADO FUNERAL SERVICE, LLC",GRAND JUNCTION,CO,US,HOUSE,E1,HOUSE REGION,G1,HOUSE ACCOUNTS,CO,,NaN,O,,,House,House,,NaN,FH,FP,973.0,300,300,300,,8014,CSC DENVER INVENTORY,EVERY 2 WEEK DELIVERY,Bi-Weekly,0,Non-Buyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,dba CALLANHAN-EDFAST MORTUARY & CREM,2515 PATTERSON RD,,81505-3610
327,149608,1785 E.E. STRINGER FUNERAL HOME,149608,1785 E.E. STRINGER FUNERAL HOME,KIRBYVILLE,TX,US,HOUSE,E1,HOUSE REGION,G1,HOUSE ACCOUNTS,CO,SCI,SERVICE CORPORATION INT'L,SCI,,,SCI,House,,NaN,FH,FP,490.0,240,240,240,,8001,CSC HOUSTON INVENTORY,MWF,3 x per week,0,Non-Buyer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,111 S MARGARET,,,75956-2131
424,880895,AZADA ADAIR FUNERAL HOMES - AVALON,344476,"FLFPG FOUNDAT